# Thread Basics

In this lesson, we look at the `threading` module, and think about threads at a low-level.  In other lessons, higher level abstractions around the basics are addressed.

Note that underneath `threading` lives an even lower-level module `_thread`.  As the initial underscore suggests, `_thread` is really only for developers of Python itself, or perhaps custom low-level libraries.  Use `threading` or "above."

In [1]:
from time import sleep
from random import choice, random, randint
import threading
from threading import Thread

In interface, a thread constructor looks like this:

```python
class Thread:
    def __init__(self, target, name=None, args=(), kwargs={}):
        pass
```

The `target` argument is the function that will be executed in that particular thread.  Alternately, a subclass of `Thread` might define a `.run()` method rather than using a passed in function; passing in an executable is generally more flexible.

Once a thread has been _created_ (instantiated), we need to `start()` it in order for it to begin to process.

## Basic example of a thread

In [2]:
def worker():
    print('hello', flush=True)
    time.sleep(2)
    print('world', flush=True)

In [3]:
t1 = Thread(target=worker)

In [4]:
t1.start()

hello


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/davidmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/davidmertz/miniconda3/envs/INE/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-1baa9f54510e>", line 3, in worker


## Running multiple threads concurrently

Although you *can* launch one thread as an indirect way to run one function, the reason you want to is to allow concurrency of multiple threads.

In [5]:
def worker(favcolor=None):
    sleep(random() * 5)
    value = randint(0, 99)
    name = threading.current_thread().name
    print(f'My name is {name}; '
          f'computation is {value}; '
          f'favorite color is {favcolor}')

NameError: name 'time' is not defined


In [6]:
worker()

My name is MainThread; computation is 93; favorite color is None


So far, this is just a function, albeit one that reports the name of the thread it runs in.  If your code never mentions `threading` or related modules, your program runs in one thread called "MainThread".  However, let us create several threads that utilize this function.

In [7]:
threads = [Thread(target=worker, 
                  name=f"thread-{i}", 
                  kwargs={'favcolor': choice(['blue', 'green', 'yellow'])}) 
           for i in range(5)]
threads

[<Thread(thread-0, initial)>,
 <Thread(thread-1, initial)>,
 <Thread(thread-2, initial)>,
 <Thread(thread-3, initial)>,
 <Thread(thread-4, initial)>]

In [8]:
for thread in threads:
    thread.start()

## Thread states

A thread is alive when it is running, but not otherwise.  There is both a time "before its birth" and a time "after its termination" and it is alive during neither of those times. Creating a thread only reserves resources like memory for it; it has to be `.start()`'ed to actually become alive and carry out actions.

In [9]:
t = Thread(target=worker)
t.is_alive()

False

In [10]:
t.start()
t.is_alive()

My name is thread-0; computation is 14; favorite color is yellow


True

In [11]:
time.sleep(5)  # Must wait for output from above...
t.is_alive()

My name is thread-1; computation is 49; favorite color is yellow
My name is Thread-5; computation is 47; favorite color is None


NameError: name 'time' is not defined

A thread that has finished can't be started again, as shown in the following example:

In [ ]:
try:
    t.start()
except Exception as err:
    print(repr(err))

## Joining threads

There is an element of the lifetime of threads that we brushed over.  Sometimes you would like to wait within the MainThread for a thread to finish; or indeed, wait within some child thread for some grandchild to finish.  For example:

In [ ]:
t = Thread(target=worker)
print("About to start...")
t.start()
print("All done")

That was not as intended. The "All done" message printed before the thread was done.  We *could* manually wait on it:

In [ ]:
t = Thread(target=worker)
print("About to start...")
t.start()
while t.is_alive():
    sleep(0.01)
print("All done")

There is a better, shorter, and more explicit way in `.join`'ing a thread.

In [ ]:
t = Thread(target=worker)
print("About to start...")
t.start()
t.join() # Don't continue until thread completes
print("All done")

## Thread identity

The thread class has two attributes that lets us identify each thread. The human-ready `name`, which we can set when we construct the thread, and the machine-oriented `ident` one.  The name is set immediately, either to a passed in value or a generated human-readable one.  The `ident` is None until the thread actually starts.

In [ ]:
t = Thread(target=worker)
print("Automatic thread name:", t.name)
print("Thread identity:", t.ident)

print("Starting thread...")
t.start()
print("Thread identity:", t.ident)

## Subclassing `Thread`

In prior examples,  we created threads is by passing a `target` function to be executed. An alternative, and object oriented, way to do it is to extend the Thread class:

In [ ]:
class MyThread(Thread):
    def __init__(self, time_to_sleep, name=None):
        super().__init__(name=name)
        self.time_to_sleep = time_to_sleep
        
    def run(self):
        ident = threading.current_thread().ident
        print(f"I am thread {self.name} (ID {ident}), "
              f"sleeping for {self.time_to_sleep} seconds.")
        sleep(self.time_to_sleep)
        print(f'Thread {self.name} exiting...')

In [ ]:
t = MyThread(2, name="Santiago")
t.start()

## Shared Data

As we'll see, Threads can access shared data within the process they live in. Example:

In [ ]:
TIME_TO_SLEEP = 2

In [ ]:
def simple_worker():
    myself = threading.current_thread()
    print(f"I am thread {myself.name}, and I'm sleeping for {TIME_TO_SLEEP}.")
    time.sleep(TIME_TO_SLEEP)
    print(f'Thread {myself.name} exiting...')

In [ ]:
t1 = Thread(target=simple_worker, name='Bubbles')
t2 = Thread(target=simple_worker, name='Blossom')
t3 = Thread(target=simple_worker, name='Buttercup')

In [ ]:
t1.start()

In [ ]:
t2.start()

In [ ]:
t3.start()

How is this possible?

Remember, all threads live **within the same process**, and the variable `TIME_TO_SLEEP` is stored in the process. So all the threads created can access that variable.

## A real example

In the `crypto-examples` directory, we've included a real example of a web server that contains prices of different cryptocurrencies. You can run it with `python flask_app.py --sleep [sleep in seconds]`. The server can be slowed down on purpose to simulate a real slow server.

Let's check how to get one price as an example:

In [ ]:
BASE_URL = "http://localhost:5000"

In [ ]:
import requests

In [ ]:
resp = requests.get(f"{BASE_URL}/price/bitfinex/btc/2020-04-01")

In [ ]:
resp

In [ ]:
resp.json()

Now, let's suppose we want to get the price of Bitcoin from 3 different exchanges: `bitfinex`, `bitstamp` and `kraken`. The sequential requests would take us 6 seconds (with a sleep param in 2).

In [ ]:
%%time
for exchange in ['bitfinex', 'bitstamp', 'kraken']:
    url = f"{BASE_URL}/price/{exchange}/btc/2020-04-01"
    resp = requests.get(url)
    print(f"{exchange.title()}: ${resp.json()['close']}")

Let's now move it to threads! For now, we'll just **print** the output, as we'll se data sharing in further lessons...

In [ ]:
%%time
def check_price(exchange, symbol, date, base_url=BASE_URL):
    "General price-checker for given exchange, symbol, and date"
    url = f"{base_url}/price/{exchange}/{symbol}/{date}"
    resp = requests.get(f"{base_url}/price/{exchange}/{symbol}/{date}")
    print(f"{exchange.title()}: ${resp.json()['close']}")

check_price('bitfinex', 'btc', '2020-04-01')

In [ ]:
threads = [
    Thread(target=check_price, args=(exchange, 'btc', '2020-04-01'))
    for exchange in EXCHANGES
]

print("Starting threads...")
%time for t in threads: t.start()
print('-----')

print("Joining threads...")
%time for t in threads: t.join()

My name is thread-4; computation is 49; favorite color is yellow
My name is thread-3; computation is 64; favorite color is green
My name is thread-2; computation is 16; favorite color is green


## How many threads can we start?

Let's say we need to get prices for 10 exchanges, 3 symbols, for a total of 30 days. 

We cannot efficiently start 900 threads at once, however. Each threads consumes resources and too many threads tend to bog down your system.

So, what can we do when we need to process too many concurrent jobs? We'll create workers and use a consumer-producer model. But first, we need to talk about shared data, race conditions and synchronization...

## Summary:

* `threading` module ✅
* `_thread`  module ❌

A thread's life cycle is Instantiated > Started > Running > Finished.